# LTA API usage
- bus stops
- bus routes
- bus services
- passenger volume by bus stops
- estimated travel times
- traffic incidents (from weather)
- traffic flow

In [3]:
from LTA_API_key import API_key
import requests
import json
import pandas as pd
import numpy as np
import math
import importlib
from bs4 import BeautifulSoup
import re
import os


import helper_functions.routing.publicTransit
importlib.reload(helper_functions.routing.publicTransit)
import helper_functions.routing.publicTransit as publicTransit

In [2]:
def get_lta_response(url,params=None):
    """ returns API response in json format
    Args:
        url (str): url of API end point
        params (dict): parameters to request
    Returns:
        dict: API response from the input url
    """
    headers = {"AccountKey": API_key}

    # A GET request to the API
    response = requests.request("GET", url, headers=headers,params=params)

    # Print the response
    return response.json()


In [ ]:
bus_stops = get_lta_response(url = "https://datamall2.mytransport.sg/ltaodataservice/BusStops")
bus_stops

In [ ]:
import pandas as pd
import os
df = pd.DataFrame(bus_stops['value'])
df.to_csv(os.path.join(r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_LTA",'SG_bus_stops.csv'),index=False)

In [ ]:
[v for v in bus_stops['value'] if v['BusStopCode']=="10009"]

In [ ]:
bus_routes = get_lta_response(url = "https://datamall2.mytransport.sg/ltaodataservice/BusRoutes")
bus_routes


In [ ]:
bus_services = get_lta_response(url = "https://datamall2.mytransport.sg/ltaodataservice/BusServices")
bus_services

# AM_Peak_Fre: Freq of dispatch for AM Peak 0630H - 0830H (range in minutes)
# AM_Offpeak_Freq: Freq of dispatch for AM Off-Peak 0831H - 1659H (range in minutes)
# PM_Peak_Freq: Freq of dispatch for PM Peak 1700H - 1900H (range in minutes)
# PM_Offpeak_Freq: Freq of dispatch for PM Off-Peak after 1900H (range in minutes)

In [17]:
passenger_vol_by_bus_stops = get_lta_response(url = "https://datamall2.mytransport.sg/ltaodataservice/PV/Bus",
                                              params = {'Date':'202501'})
passenger_vol_by_bus_stops

{'odata.metadata': 'http://datamall2.mytransport.sg/ltaodataservice/$metadata#FarecardBatch',
 'value': [{'Link': 'https://ltafarecard.s3.ap-southeast-1.amazonaws.com/202501/transport_node_bus_202501.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEMv%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaDmFwLXNvdXRoZWFzdC0xIkgwRgIhAPAtO%2FzuUHOOgYZP3EA7PO8ALF69LcdQCD3%2F%2FicVXX5oAiEAnF7Cdm4vn7TFHFuyZF8SSggPNlQtB0S9Uvcu%2B6INAAoqwgUIVBAEGgwzNDA2NDUzODEzMDQiDKSZNuUJNg82R5ECsSqfBarILl620vDLbb6ZqomLT4%2BvEoC8whmfqHsNuMVlKTJO3EXjNcS2sMokuO7eTVm9wBeh35GKwYOT3S%2BkzFc4x14LFt4qkeTwq%2BONqYgXOqEXvUIlsEQ8EEgZg%2BeVTHz5NSvziQLmG8TN8Vbi4zUfK%2F68eHe7yGd85CyvUTkigopLFjc4c%2FOTPIPYspgKkpdnQ490qydw%2BIoHWVhBTWT8IutyfVk5JZCcfKz8QOM%2BrX8oGiC099VeYTj%2BQhTXlXUfqjQlZgomt9cyptYiIxYpRQi9%2FY1dnAkGuVgXm7u0nUuCvS6Xr5rDNmrm%2B3uLgCCaF9%2Fl1IvFP6ITH4i5t9bDL9nLY6SGAAMroJs2BeIk%2Bijjlk50eWp4by%2F4ZmfwVPwJrN6y6Wm18SPkTzxgJ4aCnBFGDvCv3v72a7Zyas1Mxrge8LTu9qGMOYBW%2BuwhREcsEHzU8JR35vAPp0HyC1aigOdfyR%2BGYGKa5VzPCsY7%2BA97z%2BWkqkFhOiKWi7nRi9N

In [4]:
traffic_flow = get_lta_response(url = "https://datamall2.mytransport.sg/ltaodataservice/TrafficFlow")
traffic_flow

traffic_flow = requests.get(traffic_flow['value'][0]['Link'])
traffic_flow = traffic_flow.json()
traffic_flow
with open(os.path.join(r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_LTA\trafficflow",
                       'trafficflow_20250519.json'),'w') as f:
    json.dump(traffic_flow,f)

In [12]:
passenger_vol_by_origin_destination = get_lta_response(url = "https://datamall2.mytransport.sg/ltaodataservice/PV/ODBus",
                                                       params = {'Date':'202502'})
passenger_vol_by_origin_destination

{'odata.metadata': 'http://datamall2.mytransport.sg/ltaodataservice/$metadata#FarecardBatch',
 'value': [{'Link': 'https://ltafarecard.s3.ap-southeast-1.amazonaws.com/202502/origin_destination_bus_202502.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjELf%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaDmFwLXNvdXRoZWFzdC0xIkYwRAIgZtpwAoS%2Bxh4GwcTyqnvB9GYCpNsHdQEbHY7fe4i%2FQigCIEOvdP4OctvBHf%2BIejGiMzRzhZv44RU5DeNsxwIOlTqMKsIFCEAQBBoMMzQwNjQ1MzgxMzA0IgxaL5sDVYneY%2BeWwnsqnwXHbMTshYnq%2BiMcpEZ046%2F6LJXAMaEO1NXePUSdMtd8vpZvl2mNZw4WnLXul7x4y3qlefsZFkzJrA4XMe67b5siEJLgeH9hFoH%2BR0H1w8w8HOaap94HZE7DiJSnEqXhVTNJXfbpT2URiGrnSBx5bCaslHeZJizdiQhuXFdDU5GMyoytAmfU4MkzsB1%2F5a8afeG%2BNZI6sG0zx8CSCRqTdUMZ2lzwao%2BPFl3hUzGm7d5E%2F5%2FiE6%2FsUWmk7FrXz4Hr%2FPz0OX7j4FVh5D29pRhOCFBCd84V4TeEkaiNkHmCwVttWdybqSyz2XumOIpUS5mn0p39BpFO%2B2FRSfUH2xohyamVD3RFc0Wto8%2FqcV%2BclbtHk%2Fpem68PB2KPQ6AaGH6g%2B2Isa2scgDaYSzcOFNNKd%2Fe1Y0bRKfPatmhUNPxSu%2F5p8PMQrE8Zhj0ivwMqvRL84ImishIvnOULgCPk%2FOLbZRQzniDqOndNFgCDxP%2BQdSdsGGd29bo9B7SZmk6qj

In [ ]:
passenger_vol_by_train_stations = get_lta_response(url = "https://datamall2.mytransport.sg/ltaodataservice/PV/Train")
passenger_vol_by_train_stations

In [ ]:
estimated_travel_times = get_lta_response(url = "https://datamall2.mytransport.sg/ltaodataservice/EstTravelTimes")
estimated_travel_times

In [ ]:
pd.DataFrame(estimated_travel_times['value']).to_csv(r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_LTA\20250319_estimatedTravelTimes.csv",index=False)

In [22]:
travel_speed_bands = get_lta_response(url = "https://datamall2.mytransport.sg/ltaodataservice/v3/TrafficSpeedBands")
travel_speed_bands

{'odata.metadata': 'http://datamall2.mytransport.sg/ltaodataservice/$metadata#TrafficSpeedBands',
 'lastUpdatedTime': '2025-04-21 16:25:00',
 'value': [{'LinkID': '103000000',
   'RoadName': 'KENT ROAD',
   'RoadCategory': 'E',
   'SpeedBand': 3,
   'MinimumSpeed': '20',
   'MaximumSpeed': '29',
   'StartLon': '103.85298052044503',
   'StartLat': '1.3170142376560023',
   'EndLon': '103.85259882242372',
   'EndLat': '1.3166840028663076'},
  {'LinkID': '103000010',
   'RoadName': 'BUCKLEY ROAD',
   'RoadCategory': 'E',
   'SpeedBand': 8,
   'MinimumSpeed': '70',
   'MaximumSpeed': '999',
   'StartLon': '103.84102305136321',
   'StartLat': '1.3166507852203482',
   'EndLon': '103.84022564204443',
   'EndLat': '1.316912438354752'},
  {'LinkID': '103000011',
   'RoadName': 'BUCKLEY ROAD',
   'RoadCategory': 'E',
   'SpeedBand': 2,
   'MinimumSpeed': '10',
   'MaximumSpeed': '19',
   'StartLon': '103.84022564204443',
   'StartLat': '1.316912438354752',
   'EndLon': '103.84102305136321',
   'E

In [23]:
pd.DataFrame(travel_speed_bands['value']).to_csv(r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_LTA\TravelSpeedBands_20250421_16-30-00.csv",index=False)

In [ ]:
GeospatialWholeIsland = get_lta_response(url = "https://datamall2.mytransport.sg/ltaodataservice/GeospatialWholeIsland",
                                         params={'ID':'BusStopLocation'})
GeospatialWholeIsland

In [ ]:
carparkAvailability = get_lta_response(url = "https://datamall2.mytransport.sg/ltaodataservice/CarParkAvailabilityv2")
carparkAvailability

In [ ]:
import time
from datetime import datetime

time_cum = 0
while time_cum < 24*3600:
    carparkAvailability = get_lta_response(url = "https://datamall2.mytransport.sg/ltaodataservice/CarParkAvailabilityv2")
    df = pd.DataFrame(carparkAvailability['value'])
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H-%M-%S")
    print("Current Time =", current_time)
    df.to_csv(os.path.join(r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_LTA\CarParkAvailability",f'{current_time}.csv'),index=False)
    time.sleep(3600)
    time_cum += 3600
    

# GTFS Data

Current static GTFS obtained from [LTA](https://www.transit.land/feeds/f-w21z-lta)

In [ ]:
import pandas as pd
import numpy as np
import math

## shapes.txt

In [ ]:
gtfs_df = pd.read_csv(r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_LTA\gtfs-feed-lta\shapes.txt")
gtfs_df = gtfs_df.sort_values(by=['shape_id','shape_pt_sequence'])
gtfs_df_dict = {shape_id:df for shape_id, df in gtfs_df.groupby('shape_id', as_index = False)}
gtfs_df_dict

In [ ]:
def planar_distance(lat1, lon1, lat2, lon2):
    # Conversion factors
    meters_per_degree_lat = 111320  # Approximate meters per degree of latitude
    meters_per_degree_lon = 111320 * math.cos(math.radians(lat1))  # Approximate meters per degree of longitude at given latitude

    # Calculate differences in coordinates
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1

    # Convert differences to meters
    delta_lat_meters = delta_lat * meters_per_degree_lat
    delta_lon_meters = delta_lon * meters_per_degree_lon

    # Use Pythagorean theorem to calculate distance
    distance = math.sqrt(delta_lat_meters ** 2 + delta_lon_meters ** 2)
    return distance

def get_shape_dist_traveled(df):
    """ adds the shape_dist_traveled column to df - cummulative distance travelled
    Args:
        df (pd.DataFrame): GTFS dataframe of a unique shape_id. Must have lat and lon columns
    Returns:
        pd.DataFrame: with an appended column 'shape_dist_traveled' which describes the cummulative distance
    """
    coordinates = df.iloc[:,1:3].values
    # print(df.iloc[:,1:3])
    meters_per_degree_lat = 111320  # Approximate meters per degree of latitude
    # meters_per_degree_lon = 111320 * math.cos(math.radians(lat1))  # Approximate meters per degree of longitude at given latitude
    diff_coords = np.diff(coordinates,axis=0)
    distance_multiplier = np.ones(diff_coords.shape) # to perform matrix multiplication later
    distance_multiplier[:,0] = distance_multiplier[:,0]*meters_per_degree_lat
    # Approximate meters per degree of longitude at given latitude
    distance_multiplier[:,1] = distance_multiplier[:,1]*np.cos(np.radians(diff_coords[0,0]))*meters_per_degree_lat
    distance_metres = np.multiply(diff_coords,distance_multiplier)
    distance_metres = np.linalg.norm(distance_metres,axis=1)

    # calculate cummulative distance
    cum_dist = np.cumsum(distance_metres)
    cum_dist = [0] + cum_dist.tolist()
    # lin_dist = [0] + distance_metres.tolist()
    df['shape_dist_traveled'] = cum_dist
    return df

get_shape_dist_traveled(gtfs_df_dict[list(gtfs_df_dict)[-1]])

In [ ]:
shape_dist_traveled = pd.concat([get_shape_dist_traveled(df) for df in gtfs_df_dict.values()])
shape_dist_traveled.to_csv(r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_LTA\gtfs-feed-lta\shapes1.txt",index=False)
shape_dist_traveled

### stops.txt
[GTFS reference](https://gtfs.org/documentation/schedule/reference/#stopstxt)

In [ ]:
gtfs_stops = pd.read_csv(r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_LTA\gtfs-feed-lta - Copy\stops.txt")
gtfs_stops['parent_station'] = ''
rearranged_columns = ['stop_id','stop_code','stop_name','stop_lat','stop_lon','stop_url','parent_station','wheelchair_boarding']
gtfs_stops = gtfs_stops[rearranged_columns]
gtfs_stops.to_csv(r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_LTA\gtfs-feed-lta - Copy\stops1.txt")
gtfs_stops

# Scrap bus service routes

- Some missing route information in the GTFS dataset

In [ ]:
url = "https://moovitapp.com/singapore_%E6%96%B0%E5%8A%A0%E5%9D%A1-1678/lines/en-gb?ref=Lines&customerId=4908"
def scrap_moovit_lines(url):
    response = requests.get(url)
    response.text
    # Check if request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all <h3> elements inside the list
        lines = soup.find_all("span",class_="line-title primary")
        for line in lines:
            print(line.text.strip())

    return
scrap_moovit_lines(url)

In [ ]:
re.findall("SBS Transit(.*?)Changi Airport",response.text)

In [ ]:
url = "https://moovitapp.com/index/en-gb/public_transportation-line-859-Singapore_%E6%96%B0%E5%8A%A0%E5%9D%A1-1678-873544-60310654-0"
def scrap_SMRT_busRoutes(url,shape_id):
    gtfs_913T = ['46009','46281','46391','46761','46771','46781','46791','46811','46821','46831','46841','46799','46789','46779','46769','46399','46289','46008','46291',
            '46199','46181','46071','46011','46511','46521','46579','46589','46019','46079','46189','46191','46299','46009']
    
    # fetch coordinates from onemap API
    gtfs_913T_df = []
    for row_ix, busStop in enumerate(gtfs_913T):
        location = f'{busStop} (BUS STOP)'
        url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={location}&returnGeom=Y&getAddrDetails=Y"
        
        response = publicTransit.get_OneMap_response(url,headers=headers)
        response_first_result = response['results'] # get first item in the list
        road_name = response_first_result[0]['ROAD_NAME']
        shape_pt_lat = response_first_result[0]['LATITUDE']
        shape_pt_lon = response_first_result[0]['LONGITUDE']
        gtfs_913T_df.append({'busStop':busStop,'busDescription':road_name,
                             'shape_id':shape_id,'shape_pt_sequence':int(row_ix+1),
                             'shape_pt_lat':shape_pt_lat,'shape_pt_lon':shape_pt_lon})

    return pd.DataFrame(gtfs_913T_df)
# scrap_SMRT_busRoutes(url,"shape_id")

missing_gtfs_shape = scrap_SMRT_busRoutes(url,shape_id='913T:WD:0_shape')
routeId = '913T'
tripDirection = 0
missing_gtfs_shape.to_csv(os.path.join(r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_LTA\missing_gtfs_shape",
                                       f"gtfs_shape_{routeId}_{tripDirection}.csv"),index=False)
missing_gtfs_shape

In [ ]:
location = '46009 (BUS STOP)'
url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={location}&returnGeom=Y&getAddrDetails=Y"
    
response = publicTransit.get_OneMap_response(url,headers=headers)
response

In [ ]:
url = "https://moovitapp.com/index/en-gb/public_transportation-line-359-Singapore_%E6%96%B0%E5%8A%A0%E5%9D%A1-1678-904138-586713-0"
def scrap_moovit_busRoutes(url,shape_id):
    stop_list = []
    # Send an HTTP request to the website
    response = requests.get(url)
    response.text
    # Check if request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all <h3> elements inside the list
        stops = soup.select("ul.stops-list h3")
        
        # Extract and print the text from each <h3>
        for i,stop in enumerate(stops):
            busName = stop.text.strip()
            try:
                m = re.findall(r'\((.*?)\)',busName)[0]
            except:
                print(busName)
                raise Exception('bus stop ID not found')
                # m = '77009'
            stop_list.append({'busStop':m,'busDescription':busName,
                              'shape_id':shape_id,'shape_pt_sequence':int(i+1)})
    else:
        print("Failed to retrieve the webpage.")
    return pd.DataFrame(stop_list)

# missing_gtfs_shape = scrap_moovit_busRoutes(url,shape_id='359:WD:0_shape')
# missing_gtfs_shape

add coordinates of bus stop using OneMap API

In [ ]:
headers = publicTransit.generate_OneMap_token()

In [ ]:
routeId = "900"
tripDirection = 0
# scrap moovit app
url = "https://moovitapp.com/index/en-gb/public_transportation-line-900-Singapore_%E6%96%B0%E5%8A%A0%E5%9D%A1-1678-775181-575962-0"
missing_gtfs_shape = scrap_moovit_busRoutes(url,shape_id=f'{routeId}:WD:{tripDirection}_shape')
# fetch coordinates from onemap API
lat_list = []
lon_list = []
for row_ix, row in missing_gtfs_shape.iterrows():
    location = f'{row["busStop"]} (BUS STOP)'
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={location}&returnGeom=Y&getAddrDetails=Y"
    
    response = publicTransit.get_OneMap_response(url,headers=headers)
    response_first_result = response['results'] # get first item in the list
    shape_pt_lat = response_first_result[0]['LATITUDE']
    shape_pt_lon = response_first_result[0]['LONGITUDE']
    lat_list.append(shape_pt_lat)
    lon_list.append(shape_pt_lon)

missing_gtfs_shape['shape_pt_lat'] = lat_list
missing_gtfs_shape['shape_pt_lon'] = lon_list

missing_gtfs_shape.to_csv(os.path.join(r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_LTA\missing_gtfs_shape",
                                       f"gtfs_shape_{routeId}_{tripDirection}.csv"),index=False)
missing_gtfs_shape